In [2]:
import pandas as pd
import numpy as np
#import sframe
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model
import functools
import json
from math import sqrt
from __future__ import division

In [3]:
product = pd.read_csv("/Users/wxu/workspace/ML-UW-Coursera/classification/week2/amazon_baby_subset.csv",header=0)
imptWord = json.load(open("/Users/wxu/workspace/ML-UW-Coursera/classification/week2/important_words.json","r"))

In [4]:
def remove_punctuation(text):
    import string
    return text.translate(None,string.punctuation)

In [5]:
#data cleaning for review
product = product.fillna({"review":''})
product["review_clean"] = product["review"].apply(remove_punctuation)

In [6]:
#data cleaning for count important word
for word in imptWord:
    product[word] = product['review_clean'].apply(lambda s : s.split().count(word))

In [7]:
product["contain_perfect"] = product["perfect"].apply(lambda perfect: 1 if perfect>=1 else 0)
(product["contain_perfect"]==1).sum()#2955

2955

In [8]:
def get_numpy_data(df,features,label):
    # df: a data frame that need to be transformed
    # features: a list of strings for the features name
    # label : single string for the class labels
    df["constant"] = 1
    features = ['constant'] + features
    features_frame = df[features]
    features_matrix = features_frame.as_matrix()
    label_sarray = df[label]
    label_array= label_sarray.as_matrix()
    return(features_matrix,label_array)

In [9]:
wordFeature,mySentiment = get_numpy_data(product,imptWord,"sentiment")

In [25]:
def predict_prob(feature_matrix,coefficient):
    dotProduct = np.dot(feature_matrix,coefficient)
    prob = 1/(1+np.exp(-dotProduct))
    return prob

In [11]:
def feature_derivative(feature,errors):
    derivative = np.dot(feature,errors)
    return(derivative)

In [12]:
def compute_log_likelihood(feature_matrix, sentiment, coefficients):
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
    lp = np.sum((indicator-1)*scores - np.log(1. + np.exp(-scores)))
    return lp

In [26]:
def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in xrange(max_iter):
        # Predict P(y_i = +1|x_1,w) using your predict_probability() function
        # YOUR CODE HERE
        predictions = predict_prob(feature_matrix,coefficients)

        # Compute indicator value for (y_i = +1)
        indicator = (sentiment==+1)

        # Compute the errors as indicator - predictions
        errors = indicator - predictions

        for j in xrange(len(coefficients)): # loop over each coefficient
            # Recall that feature_matrix[:,j] is the feature column associated with coefficients[j]
            # compute the derivative for coefficients[j]. Save it in a variable called derivative
            # YOUR CODE HERE
            derivative = feature_derivative(feature_matrix[:,j],errors)

            # add the step size times the derivative to the current coefficient
            # YOUR CODE HERE
            coefficients[j] = coefficients[j]+step_size*derivative

        # Checking whether log likelihood is increasing
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood(feature_matrix, sentiment, coefficients)
            print 'iteration %*d: log likelihood of observed labels = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, lp)
      
    return coefficients

In [27]:
myCoef = logistic_regression(feature_matrix=wordFeature, sentiment=mySentiment, initial_coefficients=[0.0]*194, step_size=1e-7, max_iter=301)

iteration   0: log likelihood of observed labels = -36780.91768478
iteration   1: log likelihood of observed labels = -36775.13434712
iteration   2: log likelihood of observed labels = -36769.35713564
iteration   3: log likelihood of observed labels = -36763.58603240
iteration   4: log likelihood of observed labels = -36757.82101962
iteration   5: log likelihood of observed labels = -36752.06207964
iteration   6: log likelihood of observed labels = -36746.30919497
iteration   7: log likelihood of observed labels = -36740.56234821
iteration   8: log likelihood of observed labels = -36734.82152213
iteration   9: log likelihood of observed labels = -36729.08669961
iteration  10: log likelihood of observed labels = -36723.35786366
iteration  11: log likelihood of observed labels = -36717.63499744
iteration  12: log likelihood of observed labels = -36711.91808422
iteration  13: log likelihood of observed labels = -36706.20710739
iteration  14: log likelihood of observed labels = -36700.5020

In [34]:
coefficients = list(myCoef[1:]) # exclude intercept
word_coefficient_tuples = [(word, coefficient) for word, coefficient in zip(imptWord, coefficients)]
word_coefficient_tuples_pos = sorted(word_coefficient_tuples, key=lambda x:x[1], reverse=True)
word_coefficient_tuples_neg = sorted(word_coefficient_tuples, key=lambda x:x[1], reverse=False)

In [35]:
word_coefficient_tuples_pos[0:10]

[(u'great', 0.066546084170457695),
 (u'love', 0.065890762922123258),
 (u'easy', 0.06479458680257838),
 (u'little', 0.045435626308421372),
 (u'loves', 0.044976401394906038),
 (u'well', 0.030135001092107074),
 (u'perfect', 0.029739937104968462),
 (u'old', 0.020077541034775381),
 (u'nice', 0.018408707995268992),
 (u'daughter', 0.01770319990570169)]

In [36]:
word_coefficient_tuples_neg[0:10]

[(u'would', -0.053860148445203121),
 (u'product', -0.041511033392108897),
 (u'money', -0.038982037286487109),
 (u'work', -0.033069515294752737),
 (u'even', -0.030051249236035808),
 (u'disappointed', -0.028978976142317068),
 (u'get', -0.028711552980192581),
 (u'back', -0.027742697230661331),
 (u'return', -0.026592778462247283),
 (u'monitor', -0.024482100545891717)]

In [ ]:
# sort a tuple
# sorted(tuples,key = lambda x:x[1])